In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.chrome.service import Service


s = Service("C:/path/to/chromedriver.exe")
driver = webdriver.Chrome(service=s)

url = 'https://sena.territorio.la/index.php?login=true'
driver.get(url)

user='1095910391'
passwd='1095910391'

input_user=driver.find_element(By.XPATH,'/html/body/div[3]/div/div/div/form/table/tbody/tr[5]/td/input')
input_passwd=driver.find_element(By.XPATH,'/html/body/div[3]/div/div/div/form/table/tbody/tr[7]/td/label/input')

input_user.send_keys(user)
input_passwd.send_keys(passwd)

boton = driver.find_element(By.XPATH,'//*[@id="ingresar"]')
boton.click()

time.sleep(2)
course = driver.find_element(By.XPATH,'/html/body/div[12]/div/div[2]/div[1]/div[3]/div[2]/ul/li[1]/div/div[2]/span[1]/div/a')
course.click()

time.sleep(2)
evidencia = driver.find_element(By.XPATH,'/html/body/div[15]/table[1]/tbody/tr/td[1]/div/ul/li/ul/li[7]/a')
evidencia.click()

resultados = {}
for i in range(1, 165):  # Iterar N veces
    time.sleep(4)
    titulo = driver.find_element(By.XPATH, f'/html/body/div[15]/table[1]/tbody/tr/td[2]/div[3]/div[3]/div[1]/div[4]/div[{i}]/table/tbody/tr/td[1]/a').text
    calificacion = driver.find_element(By.XPATH, f'/html/body/div[15]/table[1]/tbody/tr/td[2]/div[3]/div[3]/div[1]/div[4]/div[{i}]/table/tbody/tr/td[4]/span').text
    resultados[titulo] = calificacion




In [58]:
import pandas as pd
datos = pd.DataFrame(resultados.items(), columns=['Titulo', 'Calificacion'])

In [59]:
# Establecer el ancho de las columnas en 100
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
# crear un ejemplo de dataframe
df = datos

In [62]:
# agregar columnas nuevas para el título corto y el código GA
df['Tarea'] = ''
df['Evidencia'] = ''

for i in range(len(df)):
    if pd.isna(df.loc[i, 'Titulo']):
        continue # saltar la iteración si el valor es NaN
    # verificar si el indicador "GA" está presente en el texto
    if 'GA' in df.loc[i, 'Titulo']:
        # separar el texto antes del indicador "GA"
        titulo_corto, codigo_ga = df.loc[i, 'Titulo'].split('GA')
        df.loc[i, 'Tarea'] = titulo_corto.strip()
        df.loc[i, 'Evidencia'] = 'GA' + codigo_ga.strip()
    else:
        # si el indicador "GA" no está presente, copiar todo el texto en "Titulo_corto"
        df.loc[i, 'Tarea'] = df.loc[i, 'Titulo']

In [71]:
evidencia = df.loc[:, ['Tarea', 'Evidencia', 'Calificacion']]

In [72]:
# Exportar el dataframe a un archivo de Excel
evidencia.to_excel('ejemplo.xlsx', index=False)

In [74]:
import psycopg2

# Conectarse a la base de datos
conn = psycopg2.connect(
    host="127.0.0.1",
    database="pcl",
    user="postgres",
    password="123456"
)

# Abrir un cursor
cur = conn.cursor()

# Iterar sobre las filas del dataframe y guardar los datos en la base de datos
for index, row in evidencia.iterrows():
    tarea = row['Tarea']
    evidencia = row['Evidencia']
    calificacion = row['Calificacion']
    query = f"INSERT INTO adso.scraping_web (Tarea, Evidencia, Calificacion) VALUES ('{tarea}', '{evidencia}', '{calificacion}')"
    cur.execute(query)

# Guardar los cambios y cerrar la conexión
conn.commit()
cur.close()
conn.close()



In [ ]:
#Mysql
import mysql.connector

# Conectarse a la base de datos
cnx = mysql.connector.connect(
    host="tu_host",
    user="tu_usuario",
    password="tu_contraseña",
    database="tu_basedatos"
)

# Abrir un cursor
cursor = cnx.cursor()

# Iterar sobre las filas del dataframe y guardar los datos en la base de datos
for index, row in evidencia.iterrows():
    tarea = row['Tarea']
    evidencia = row['Evidencia']
    calificacion = row['Calificacion']
    query = "INSERT INTO nombre_tabla (Tarea, Evidencia, Calificacion) VALUES (%s, %s, %s)"
    data = (tarea, evidencia, calificacion)
    cursor.execute(query, data)

# Guardar los cambios y cerrar la conexión
cnx.commit()
cursor.close()
cnx.close()
